# This notebook is responsible for the initial setup - clean all files from previous runs, load client configurations and save them

In [ ]:
import sys
import pandas as pd
import shutil
import os
import glob

from omegaconf import OmegaConf

from saiva.model.shared.load_raw_data import fetch_training_data, fetch_training_cache_data
from saiva.model.shared.database import DbEngine
from saiva.model.shared.constants import saiva_api, LOCAL_TRAINING_CONFIG_PATH
from saiva.training.utils import load_config

from eliot import to_file
to_file(sys.stdout)

### =============== Delete all files related to previous model run/training ==================

In [ ]:
file_types = ['*.pickle', '*.json', '*.csv', '*.png', '*.txt']

for file_type in file_types:
    files = glob.glob(file_type)
    for file in files:
        print(f"Removed {file}")
        os.remove(file)

### =============== Delete all files inside /data/processed & /data/raw ==================

In [ ]:
import shutil
from pathlib import Path

processed_path = Path('/data/processed')
raw_path = Path('/data/raw')
shutil.rmtree(processed_path)
shutil.rmtree(raw_path)
print("Success.......")

### ====================== Clean Configuration ======================

In [ ]:
shutil.rmtree(f'{LOCAL_TRAINING_CONFIG_PATH}generated', ignore_errors=True)
os.makedirs(f'{LOCAL_TRAINING_CONFIG_PATH}generated/', exist_ok=True)

## Load config

In [ ]:
config = load_config(LOCAL_TRAINING_CONFIG_PATH)
training_config = config.training_config

In [ ]:
print("TRAIN_START_DATE:", training_config.training_metadata.experiment_dates.train_start_date)
print("TEST_END_DATE:", training_config.training_metadata.experiment_dates.test_end_date)
print("CLIENTS:", [organization_config.organization_id for organization_config in training_config.organization_configs_setup])

### ====================== Load ml model organization configs ======================

In [ ]:
organization_configs = []

for organization_config in training_config.organization_configs_setup:
    datasource = saiva_api.data_sources.get(datasource_id=organization_config.datasource_id)
    datasource.source_database_credentials_secret_id = 'dev-sqlserver'
    organization_configs.append({
        'organization_id': organization_config.organization_id,
        'datasource': datasource
    })

### Store the configuration to be able to change the configuration manually

In [ ]:
conf = OmegaConf.create({'training_config': {'organization_configs': organization_configs}})
OmegaConf.save(conf, f'{LOCAL_TRAINING_CONFIG_PATH}generated/organization_configs.yaml')